In [1]:
import os
from glob import glob
from osgeo import gdal


In [2]:
ref_tile = 176

In [13]:
# Multiband image Tif
in_path = f'/projects/my-private-bucket/HLS-1DCNN-AGB/data/tif/HLS_composites/monthly/br_af_grid60km_prj_evi2_max/tile_{ref_tile:03d}/'
in_path


'/projects/my-private-bucket/HLS-1DCNN-AGB/data/tif/HLS_composites/monthly/br_af_grid60km_prj_evi2_max/tile_176/'

In [14]:
start_year, end_year = 2018, 2022


In [15]:
# Collect tif files by year
tif_files = []
for year in range(start_year, end_year + 1):
    year_dir = f'{in_path}/{str(year)}'
    # print(year_dir)
    # print(glob(f'{year_dir}/*.tif'))
    if os.path.exists(year_dir):
        tif_files.extend(glob(f'{year_dir}/*.tif'))

In [16]:
len(tif_files)

60

In [17]:
# Create ref name to save outputs using one path
fname = os.path.basename(tif_files[0])
fn_parts = fname.replace(".tif", "").split("_")
# Example: ['HLS','89','01-01','12-30','2022','2022','percentile95.0evi2','Blue']
ref_name = f"{fn_parts[0]}.{fn_parts[1]}.{fn_parts[6]}"

ref_name

# # Create output directory named by year range
# parent_out_dir = os.path.join(
#     os.path.dirname(in_path),
#     f'bands_vrt_{start_year}-{end_year}'
# )
# os.makedirs(parent_out_dir, exist_ok=True)

'HLS.176.maxevi2'

In [43]:
for tif in tif_files:
    base = os.path.splitext(os.path.basename(tif))[0]
    # Subdir for this tif
    tif_out_dir = os.path.join(parent_out_dir, f"{base}_vrts")
    os.makedirs(tif_out_dir, exist_ok=True)

    ds = gdal.Open(tif)
    for b in range(1, ds.RasterCount + 1):
        band = ds.GetRasterBand(b)
        desc = band.GetDescription()
        if not desc:
            desc = f"band{b:02d}"
        safe_name = desc.replace(" ", "_").replace("/", "_")
        
        out_vrt = os.path.join(tif_out_dir, f"{base}_{safe_name}.vrt")
        gdal.Translate(out_vrt, tif, format="VRT", bandList=[b])
        print(f"Created: {out_vrt}")
    ds = None

Created: /projects/my-private-bucket/HLS-1DCNN-AGB/data/tif/HLS_composites/monthly/br_af_grid30km_prj_evi2_p95/tile_089/bands_vrt_2020-2022/HLS_89_01-01_01-31_2020_2020_percentile95.0evi2_vrts/HLS_89_01-01_01-31_2020_2020_percentile95.0evi2_Blue.vrt
Created: /projects/my-private-bucket/HLS-1DCNN-AGB/data/tif/HLS_composites/monthly/br_af_grid30km_prj_evi2_p95/tile_089/bands_vrt_2020-2022/HLS_89_01-01_01-31_2020_2020_percentile95.0evi2_vrts/HLS_89_01-01_01-31_2020_2020_percentile95.0evi2_Green.vrt
Created: /projects/my-private-bucket/HLS-1DCNN-AGB/data/tif/HLS_composites/monthly/br_af_grid30km_prj_evi2_p95/tile_089/bands_vrt_2020-2022/HLS_89_01-01_01-31_2020_2020_percentile95.0evi2_vrts/HLS_89_01-01_01-31_2020_2020_percentile95.0evi2_Red.vrt
Created: /projects/my-private-bucket/HLS-1DCNN-AGB/data/tif/HLS_composites/monthly/br_af_grid30km_prj_evi2_p95/tile_089/bands_vrt_2020-2022/HLS_89_01-01_01-31_2020_2020_percentile95.0evi2_vrts/HLS_89_01-01_01-31_2020_2020_percentile95.0evi2_NIR.vrt
C